In [1]:
!RNAdistance --version
#RNAdistance 2.3.0cuda

'RNAdistance' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
import os
import subprocess
import numpy as np
import pandas as pd
from tqdm import tqdm

In [5]:
DATA_PATH = "./data/"

In [7]:
raw_fn = "archiveII_220808_rnafold"
dref_all = pd.read_csv(DATA_PATH + raw_fn + ".csv", index_col = 1)
display(dref_all.head(3))
display(dref_all.shape)

,id
folding,
.(((((((((...(.(((((((.((.((.(((((.......))))).....)).)).))))))).).((.......((((.......)))).......)).)))))))))..,5s_Acholeplasma-laidlawii-1
.(((((((((.....((((((((....((((...(((...........))).))))...)))).))))(((.........((((....)))).........)))..))))))))),5s_Acidovorax-temperans-1
(((((((....(((.((((........((((((((.....)))..)))))...(((.........)))((((((....))))))................((((((((..((((.........))))))))))))...(((((.....((((((...((((((......))))))...)))))).......)))))..(((((..((((((....))))))..)))))..(((((((((.(((...........))).)))))))))..............(((((((....((((((..(((....)))..))))))..))))))))))).)))...)))))))....,tmRNA_Stre.gord._TRW-29390_1-349


(3975, 1)

In [8]:
#dref = dref_all.head(10) # testing
dref = dref_all

Results = pd.DataFrame(np.zeros((len(dref.index),len(dref.index))))-1
Results.index = dref.index
Results.columns = dref.index
for i,rowi in tqdm(dref.iterrows(),total=len(dref)):
    for j,rowj in dref.iterrows():
        if Results.loc[j,i] < 0:
            echo_line = rowi.structure + "\n" + rowj.structure
            # RNAdistance cannot handle pseudoknots (treated as unpaired) https://pubmed.ncbi.nlm.nih.gov/36077037/
            echo_line = echo_line.replace('<','.').replace('>','.').replace('{','.').replace('}','.')
            oo = subprocess.check_output(["RNAdistance"], input=echo_line.encode('utf-8'))

            if oo != b"":
                max_len = len(rowi.structure) if len(rowi.structure)>len(rowj.structure) else len(rowj.structure) 
                Results.loc[i,j] = float(oo[2:])/max_len
            else:
                Results.loc[i,j] = np.nan
                print("Warning: RNAdistance error with {} and {}".format(i, j))
        else:
            Results.loc[i,j] = Results.loc[j,i]

  0%|          | 0/3975 [00:00<?, ?it/s]


AttributeError: 'Series' object has no attribute 'structure'

In [9]:
Results.to_csv("data/rnadist_f_all.csv")

In [10]:
Results.to_hdf("data/rnadist_f_all.h5", key='rnadist', mode='w')

ImportError: Missing optional dependency 'pytables'.  Use pip or conda to install pytables.